Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0,01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos o valor zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import random

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))



In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO: # nao queremos que ele nos de individuos que nao resolvam o problema
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")


# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

Maior valor: 13400 | Peso: 8.6
Maior valor: 13550 | Peso: 11.6
Maior valor: 14000 | Peso: 11.5
Maior valor: 16400 | Peso: 12.1

Você deve pegar os seguintes itens:
+ Harry Potter: ele voltou, confia!
+ Quadrinho super raro do Aranha-Homem da vida real
+ tablet
+ teclado musical
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 16400 dinheiros e peso de 12.1 unidades de massa.


#### discusão

Na hora de fazer os individuos, eles representam a informação do que tem no dicionário(mochila), para representar, usmaos as caixa binária, sendo que o 1 representa estar na mochila e o 0 não. Assim, usamos uma técninca simples, para resolver algo mais complexo e com maior relevância. 

Na hora da função objetivo, tivemos que pensar em uma punição para os itens que passassem o valor máximo da mochila e, como é um problema de maximização, esse valores ganham valores pequenos, assim esses objetos tem menor chance de entrar na nossa mochila.


Gostariamos de gerar a menor quantidade de individuos invalidos possíveis





## Conclusão



Para resolver o problema da mochila aplicando restrições na busca, que consiste em descobrir quais são os possíveis objetos que podem ser colocados na mochila, sem que passe o peso máximo e o valor seja máximo possível, dentro dos objetos aleatórios que foram sorteados, utilizamos algoritmos genéticos simples com algumas variações, sendo elas: criação de uma função que computa o preço e peso total da mochila, para não precisar reescrever todas as vezes; função objetivo do indivíduo que, como discutido, coloca uma penalidade para os itens que extrapolam o peso da mochila. Assim conseguimos chegar no melhor valor sem passar o peso limite. 




## Playground



hemorragia -
idade gestacional -
dor - escala numérica
irresponsíva  - 
sinais de choque - 
síndrome hipertensiva - 

In [ ]:
mulheres = {
    'mulher1': {
        'hemorragia': 78,
        'idade gestacional': 92,
        'dor': 43,
        'irresponsíva': 60,
        'sinais de choque': 88,
        'síndrome hipertensiva': 71
    },
    'mulher2': {
        'hemorragia': 65,
        'idade gestacional': 77,
        'dor': 21,
        'irresponsíva': 94,
        'sinais de choque': 33,
        'síndrome hipertensiva': 85
    },
    'mulher3': {
        'hemorragia': 50,
        'idade gestacional': 80,
        'dor': 63,
        'irresponsíva': 48,
        'sinais de choque': 92,
        'síndrome hipertensiva': 77
    },
    'mulher4': {
        'hemorragia': 70,
        'idade gestacional': 42,
        'dor': 15,
        'irresponsíva': 88,
        'sinais de choque': 55,
        'síndrome hipertensiva': 33
    },
    'mulher5': {
        'hemorragia': 85,
        'idade gestacional': 60,
        'dor': 77,
        'irresponsíva': 39,
        'sinais de choque': 71,
        'síndrome hipertensiva': 90
    },
    'mulher6': {
        'hemorragia': 30,
        'idade gestacional': 68,
        'dor': 52,
        'irresponsíva': 80,
        'sinais de choque': 49,
        'síndrome hipertensiva': 60
    },
    'mulher7': {
        'hemorragia': 63,
        'idade gestacional': 75,
        'dor': 40,
        'irresponsíva': 55,
        'sinais de choque': 78,
        'síndrome hipertensiva': 82
    },
    'mulher8': {
        'hemorragia': 92,
        'idade gestacional': 48,
        'dor': 85,
        'irresponsíva': 73,
        'sinais de choque': 66,
        'síndrome hipertensiva': 47
    },
    'mulher9': {
        'hemorragia': 55,
        'idade gestacional': 95,
        'dor': 30,
        'irresponsíva': 62,
        'sinais de choque': 83,
        'síndrome hipertensiva': 58
    },
    'mulher10': {
        'hemorragia': 75,
        'idade gestacional': 57,
        'dor': 90,
        'irresponsíva': 44,
        'sinais de choque': 68,
        'síndrome hipertensiva': 96
    },
    'mulher11': {
        'hemorragia': 88,
        'idade gestacional': 35,
        'dor': 68,
        'irresponsíva': 90,
        'sinais de choque': 40,
        'síndrome hipertensiva': 53
    },
    'mulher12': {
        'hemorragia': 42,
        'idade gestacional': 80,
        'dor': 50,
        'irresponsíva': 78,
        'sinais de choque': 91,
        'síndrome hipertensiva': 73
    },
    'mulher13': {
        'hemorragia': 67,
        'idade gestacional': 72,
        'dor': 18,
        'irresponsíva': 92,
        'sinais de choque': 35,
        'síndrome hipertensiva': 87
    },
    'mulher14': {
        'hemorragia': 49,
        'idade gestacional': 62,
        'dor': 74,
        'irresponsíva': 50,
        'sinais de choque': 76,
        'síndrome hipertensiva': 81
    },
    'mulher15': {
        'hemorragia': 80,
        'idade gestacional': 53,
        'dor': 32,
        'irresponsíva': 65,
        'sinais de choque': 95,
        'síndrome hipertensiva': 69
    },
    'mulher16': {
        'hemorragia': 52,
        'idade gestacional': 88,
        'dor': 58,
        'irresponsíva': 82,
        'sinais de choque': 45,
        'síndrome hipertensiva': 63
    },
    'mulher17': {
        'hemorragia': 35,
        'idade gestacional': 65,
        'dor': 45,
        'irresponsíva': 70,
        'sinais de choque': 85,
        'síndrome hipertensiva': 94
    },
    'mulher18': {
        'hemorragia': 73,
        'idade gestacional': 50,
        'dor': 83,
        'irresponsíva': 33,
        'sinais de choque': 60,
        'síndrome hipertensiva': 74
    },
    'mulher19': {
        'hemorragia': 90,
        'idade gestacional': 38,
        'dor': 60,
        'irresponsíva': 75,
        'sinais de choque': 88,
        'síndrome hipertensiva': 52
    },
    'mulher20': {
        'hemorragia': 58,
        'idade gestacional': 90,
        'dor': 55,
        'irresponsíva': 47,
        'sinais de choque': 71,
        'síndrome hipertensiva': 80
    },
    'mulher21': {
        'hemorragia': 45,
        'idade gestacional': 83,
        'dor': 38,
        'irresponsíva': 58,
        'sinais de choque': 83,
        'síndrome hipertensiva': 50
    },
    'mulher22': {
        'hemorragia': 78,
        'idade gestacional': 55,
        'dor': 91,
        'irresponsíva': 42,
        'sinais de choque': 66,
        'síndrome hipertensiva': 35
    },
    'mulher23': {
        'hemorragia': 63,
        'idade gestacional': 40,
        'dor': 30,
        'irresponsíva': 70,
        'sinais de choque': 93,
        'síndrome hipertensiva': 72
    },
    'mulher24': {
        'hemorragia': 95,
        'idade gestacional': 60,
        'dor': 80,
        'irresponsíva': 37,
        'sinais de choque': 51,
        'síndrome hipertensiva': 77
    },
    'mulher25': {
        'hemorragia': 35,
        'idade gestacional': 85,
        'dor': 62,
        'irresponsíva': 77,
        'sinais de choque': 45,
        'síndrome hipertensiva': 96
    },
    'mulher26': {
        'hemorragia': 70,
        'idade gestacional': 50,
        'dor': 78,
        'irresponsíva': 55,
        'sinais de choque': 89,
        'síndrome hipertensiva': 68
    },
    'mulher27': {
        'hemorragia': 85,
        'idade gestacional': 75,
        'dor': 55,
        'irresponsíva': 80,
        'sinais de choque': 47,
        'síndrome hipertensiva': 72
    },
    'mulher28': {
        'hemorragia': 53,
        'idade gestacional': 67,
        'dor': 39,
        'irresponsíva': 90,
        'sinais de choque': 73,
        'síndrome hipertensiva': 58
    },
    'mulher29': {
        'hemorragia': 75,
        'idade gestacional': 82,
        'dor': 48,
        'irresponsíva': 65,
        'sinais de choque': 92,
        'síndrome hipertensiva': 43
    },
    'mulher30': {
        'hemorragia': 40,
        'idade gestacional': 70,
        'dor': 65,
        'irresponsíva': 53,
        'sinais de choque': 75,
        'síndrome hipertensiva': 90
    },
    'mulher31': {
        'hemorragia': 68,
        'idade gestacional': 45,
        'dor': 92,
        'irresponsíva': 35,
        'sinais de choque': 59,
        'síndrome hipertensiva': 82
    },
    'mulher32': {
        'hemorragia': 83,
        'idade gestacional': 62,
        'dor': 73,
        'irresponsíva': 88,
        'sinais de choque': 42,
        'síndrome hipertensiva': 67
    },
    'mulher33': {
        'hemorragia': 48,
        'idade gestacional': 75,
        'dor': 55,
        'irresponsíva': 72,
        'sinais de choque': 85,
        'síndrome hipertensiva': 55
    },
    'mulher34': {
        'hemorragia': 70,
        'idade gestacional': 52,
        'dor': 80,
        'irresponsíva': 50,
        'sinais de choque': 75,
        'síndrome hipertensiva': 78
    },
    'mulher35': {
        'hemorragia': 95,
        'idade gestacional': 68,
        'dor': 35,
        'irresponsíva': 77,
        'sinais de choque': 60,
        'síndrome hipertensiva': 85
    },
    'mulher36': {
        'hemorragia': 50,
        'idade gestacional': 88,
        'dor': 58,
        'irresponsíva': 82,
        'sinais de choque': 40,
        'síndrome hipertensiva': 68
    },
    'mulher37': {
        'hemorragia': 77,
        'idade gestacional': 35,
        'dor': 45,
        'irresponsíva': 70,
        'sinais de choque': 83,
        'síndrome hipertensiva': 94
    },
    'mulher38': {
        'hemorragia': 92,
        'idade gestacional': 50,
        'dor': 83,
        'irresponsíva': 33,
        'sinais de choque': 60,
        'síndrome hipertensiva': 74
    },
    'mulher39': {
        'hemorragia': 60,
        'idade gestacional': 90,
        'dor': 55,
        'irresponsíva': 47,
        'sinais de choque': 71,
        'síndrome hipertensiva': 80
    },
    'mulher40': {
        'hemorragia': 45,
        'idade gestacional': 83,
        'dor': 38,
        'irresponsíva': 58,
        'sinais de choque': 83,
        'síndrome hipertensiva': 50
    },
    'mulher41': {
        'hemorragia': 78,
        'idade gestacional': 55,
        'dor': 91,
        'irresponsíva': 42,
        'sinais de choque': 66,
        'síndrome hipertensiva': 35
    },
    'mulher42': {
        'hemorragia': 63,
        'idade gestacional': 40,
        'dor': 30,
        'irresponsíva': 70,
        'sinais de choque': 93,
        'síndrome hipertensiva': 72
    },
    'mulher43': {
        'hemorragia': 95,
        'idade gestacional': 60,
        'dor': 80,
        'irresponsíva': 37,
        'sinais de choque': 51,
        'síndrome hipertensiva': 77
    },
    'mulher44': {
        'hemorragia': 35,
        'idade gestacional': 85,
        'dor': 62,
        'irresponsíva': 77,
        'sinais de choque': 45,
        'síndrome hipertensiva': 96
    },
    'mulher45': {
        'hemorragia': 70,
        'idade gestacional': 50,
        'dor': 78,
        'irresponsíva': 55,
        'sinais de choque': 89,
        'síndrome hipertensiva': 68
    },
    'mulher46': {
        'hemorragia': 85,
        'idade gestacional': 75,
        'dor': 55,
        'irresponsíva': 80,
        'sinais de choque': 47,
        'síndrome hipertensiva': 72
    },
    'mulher47': {
        'hemorragia': 53,
        'idade gestacional': 67,
        'dor': 39,
        'irresponsíva': 90,
        'sinais de choque': 73,
        'síndrome hipertensiva': 58
    },
    'mulher48': {
        'hemorragia': 75,
        'idade gestacional': 82,
        'dor': 48,
        'irresponsíva': 65,
        'sinais de choque': 92,
        'síndrome hipertensiva': 43
    },
    'mulher49': {
        'hemorragia': 40,
        'idade gestacional': 70,
        'dor': 65,
        'irresponsíva': 53,
        'sinais de choque': 75,
        'síndrome hipertensiva': 90
    },
    'mulher50': {
        'hemorragia': 68,
        'idade gestacional': 45,
        'dor': 92,
        'irresponsíva': 35,
        'sinais de choque': 59,
        'síndrome hipertensiva': 82
    }
}
